In [1]:
# DEMO: Load models + Predict

import os
import joblib
import pandas as pd
from IPython.display import display

# Đường dẫn tới model
RIDGE_PATH = "ridge_demand.pkl"
RF_PATH    = "rf_stockout.pkl"


# Check tồn tại file
for p in [RIDGE_PATH, RF_PATH]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Không tìm thấy file: {p}. Hãy đặt file model cùng thư mục với notebook demo hoặc sửa path.")


In [2]:
# Load model
ridge_model = joblib.load(RIDGE_PATH)
rf_model    = joblib.load(RF_PATH)

print(" - Ridge:", type(ridge_model))
print(" - RF   :", type(rf_model))

 - Ridge: <class 'dict'>
 - RF   : <class 'dict'>


In [6]:
ridge_bundle = ridge_model
rf_bundle    = rf_model

ridge_est = ridge_bundle["model"]
rf_est    = rf_bundle["model"]

ridge_scaler = ridge_bundle["scaler"]
ridge_ohe    = ridge_bundle["ohe"]

rf_scaler = rf_bundle["scaler"]
rf_ohe    = rf_bundle["ohe"]

In [7]:
import pandas as pd

def build_X_final(input_df, bundle):
    num_features = bundle["num_features"]
    bin_features = bundle["bin_features"]
    cat_features = bundle["cat_features"]
    scaler = bundle["scaler"]
    ohe = bundle["ohe"]

    # đảm bảo đủ cột 
    for c in num_features + bin_features + cat_features:
        if c not in input_df.columns:
            input_df[c] = 0

    # numeric
    X_num = scaler.transform(input_df[num_features])
    X_num_df = pd.DataFrame(X_num, columns=num_features)

    # bin
    X_bin_df = input_df[bin_features].copy()

    # cat
    X_cat = ohe.transform(input_df[cat_features])
    cat_names = ohe.get_feature_names_out(cat_features)
    X_cat_df = pd.DataFrame(X_cat, columns=cat_names)

    # concat
    X_final = pd.concat([X_num_df, X_bin_df, X_cat_df], axis=1)
    return X_final


In [8]:
input_data = {
    # Numeric / Binary
    "Inventory Level": 30,
    "Units Sold": 90,
    "Units Ordered": 80,
    "Price": 70,
    "Effective_Price": 70.0,
    "Discount": 0,
    "Has_Discount": 0,
    "Promotion": 0,
    "Competitor Pricing": 72.0,
    "Epidemic": 0,
    "Promo_Epidemic": 0,
    "Month": 6,
    "DayOfWeek": 2,
    "IsWeekend": 0,

    # Categorical
    "Category": "Electronics",
    "Region": "North",
    "Weather Condition": "Sunny",
    "Seasonality": "Summer"
}

input_df = pd.DataFrame([input_data])

print("\nDữ liệu đầu vào:")
display(input_df)



Dữ liệu đầu vào:


,Inventory Level,Units Sold,Units Ordered,Price,Effective_Price,Discount,Has_Discount,Promotion,Competitor Pricing,Epidemic,Promo_Epidemic,Month,DayOfWeek,IsWeekend,Category,Region,Weather Condition,Seasonality
0,30,90,80,70,70.0,0,0,0,72.0,0,0,6,2,0,Electronics,North,Sunny,Summer


In [9]:
X_final_ridge = build_X_final(input_df_ridge.copy(), ridge_bundle)
pred_demand = ridge_est.predict(X_final_ridge)[0]
print("Demand dự đoán:", float(pred_demand))



Demand dự đoán: 109.70494881384604


In [10]:
X_final_rf = build_X_final(input_df_rf.copy(), rf_bundle)

pred_stockout = rf_est.predict(X_final_rf)[0]
proba_stockout = rf_est.predict_proba(X_final_rf)[:, 1][0]

print("Stockout dự đoán (0/1):", int(pred_stockout))
print("Xác suất Stockout:", float(proba_stockout))


Stockout dự đoán (0/1): 1
Xác suất Stockout: 0.63
